# Assignment 3

## Part 1

### A

In [1]:
import mysql.connector
from datetime import datetime

conn = mysql.connector.connect(
    host=" Noahs-Mac-mini.local",
    user="monty",
    password="sushiSQL",
    database="Assn3"
)
cursor = conn.cursor()

### B

In [2]:
create_table_query = """
CREATE TABLE IF NOT EXISTS `WikiEdit` (
    `RevisionID` INT PRIMARY KEY,
    `ArticleName` VARCHAR(500),
    `EditDate` DATE,
    `UserName` VARCHAR(50)
)
"""
cursor.execute(create_table_query)
print("complete")

complete


### C

In [3]:
file_path = "data/wiki_edit.txt"
with open(file_path, 'r') as file:
    for line_number, line in enumerate(file, start=1):
        # Split values based on whitespace
        fields = line.strip().split()

        if len(fields) == 4:
            try:
                revision_id, article_name, edit_date_str, user_name = fields

                # Convert EditDate to date object
                edit_date = datetime.strptime(edit_date_str, '%Y-%m-%d').date()

                # Insert data into the database
                insert_query = """
                INSERT INTO `WikiEdit` (`RevisionID`, `ArticleName`, `EditDate`, `UserName`)
                VALUES (%s, %s, %s, %s)
                """
                cursor.execute(insert_query, (int(revision_id), article_name, edit_date, user_name))
                conn.commit()
            except ValueError as e:
                print(f"Ignoring line {line_number}: {line.strip()} - {e}")
        else:
            print(f"Ignoring line {line_number}: {line.strip()} - Not enough values to unpack (expected 4, got {len(fields)})")

### D

In [4]:
largest_edit_query = """
SELECT ArticleName, COUNT(*) AS EditCount
FROM `WikiEdit`
GROUP BY ArticleName
ORDER BY EditCount DESC
LIMIT 1
"""
cursor.execute(largest_edit_query)
result = cursor.fetchone()
if result:
    article_name_with_most_edits = result[0]
    print(f"Article name with the largest number of edits: {article_name_with_most_edits}")

# Close the connection
cursor.close()
conn.close()

Article name with the largest number of edits: 2004_Indian_Ocean_earthquake


## PART 2

### A

In [5]:
# !brew services start mongodb/brew/mongodb-community
# !mongod --config /opt/homebrew/etc/mongod.conf
# !mongosh
# !use mongoAssn3

Service `mongodb-community` already started, use `brew services restart mongodb-community` to restart.


### B

In [9]:
# !pip install pymongo
import json
from pymongo import MongoClient
from datetime import datetime

# MongoDB connection details
mongo_uri = "mongodb://localhost:27017/"
database_name = "mongoAssn3"
followers_collection_name = "followers"
followees_collection_name = "followees"

# Connect to MongoDB
client = MongoClient(mongo_uri)
database = client[database_name]

# Function to insert data into a collection
def insert_data(collection, data):
    for entry in data:
        try:
            creation_time_str = entry.get('creation_time', '')
            if creation_time_str:
                entry['creation_time'] = datetime.strptime(creation_time_str, '%Y-%m-%d %H:%M:%S')  # Convert creation_time to datetime
            else:
                entry['creation_time'] = None  # Set to None if 'creation_time' is empty
            collection.insert_one(entry)
        except ValueError as e:
            print(f"Ignoring entry with invalid creation_time: {entry}")

# Read and insert data for followers
with open('data/followers.json', 'r', encoding='utf-8') as followers_file:
    followers_data = json.load(followers_file)
    followers_collection = database[followers_collection_name]
    insert_data(followers_collection, followers_data)

# Read and insert data for followees
with open('data/followees.json', 'r', encoding='utf-8') as followees_file:
    followees_data = json.load(followees_file)
    followees_collection = database[followees_collection_name]
    insert_data(followees_collection, followees_data)

# Close the MongoDB connection
client.close()

## C

### I

In [18]:
followers_collection_name = "followers"

# Connect to MongoDB
client = MongoClient(mongo_uri)
database = client[database_name]

# Followers count average query
average_followers_query = [
    {
        "$group": {
            "_id": None,
            "average_followers_count": {"$avg": "$followers_count"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_followers_count": {"$round": ["$average_followers_count", 2]}
        }
    }
]

# Execute the query
result = database[followers_collection_name].aggregate(average_followers_query)

# Print the result
for doc in result:
    print("Average Followers Count:", doc["average_followers_count"])

# Close the MongoDB connection
client.close()

Average Followers Count: 2983.21


### II

In [19]:
followees_collection_name = "followees"

# Connect to MongoDB
client = MongoClient(mongo_uri)
database = client[database_name]

# Followees count average query with rounding to 2 decimal places
average_followees_query = [
    {
        "$group": {
            "_id": None,
            "average_followers_count": {"$avg": "$followers_count"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_followers_count": {"$round": ["$average_followers_count", 2]}
        }
    }
]

# Execute the query
result = database[followees_collection_name].aggregate(average_followees_query)

# Print the result
for doc in result:
    print("Average Followers Count of Followees:", doc["average_followers_count"])

# Close the MongoDB connection
client.close()

Average Followers Count of Followees: 1075218.86


### III

In [33]:
# Connect to MongoDB
client = MongoClient(mongo_uri)
database = client[database_name]

# Average followees count query
average_followees_query = [
    {
        "$group": {
            "_id": None,
            "average_followees_count": {"$avg": "$friends_count"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_followees_count": {"$round": ["$average_followees_count", 2]}
        }
    }
]


# Execute the query
result = database[followers_collection_name].aggregate(average_followees_query)

# Print the result
for doc in result:
    print("Average followees count of followers:", doc['average_followees_count'])

# Close the MongoDB connection
client.close()

Average followees count of followers: 1055.49
got here 2


### IV

In [34]:
# Connect to MongoDB
client = MongoClient(mongo_uri)
database = client[database_name]

# Followees count average query with rounding to 2 decimal places
average_followees_query = [
    {
        "$group": {
            "_id": None,
            "average_followees_count": {"$avg": "$friends_count"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_followees_count": {"$round": ["$average_followees_count", 2]}
        }
    }
]

# Execute the query
result = database[followees_collection_name].aggregate(average_followees_query)

# Print the result
for doc in result:
    print("Average Followees Count of Followees:", doc["average_followees_count"])

# Close the MongoDB connection
client.close()

Average Followees Count of Followees: 1019.7


### V

In [65]:
# Connect to MongoDB
client = MongoClient(mongo_uri)
database = client[database_name]

# Number of verified followers query
verified_followers_query = [
    {
        "$match": {"verified": True}
    },
    {
        "$group": {
            "_id": None,
            "count": {"$sum": 1}
        }
    }
]

# Number of verified followees query
verified_followees_query = [
    {
        "$match": {"verified": True}
    },
    {
        "$group": {
            "_id": None,
            "count": {"$sum": 1}
        }
    }
]

# Create a temporary collection by merging followers and followees
temp_combined_collection = "temp_combined_collection"
database[followers_collection_name].aggregate([
    {"$match": {"verified": True}},
    {"$merge": {"into": temp_combined_collection}}
], allowDiskUse=True)

database[followees_collection_name].aggregate([
    {"$match": {"verified": True}},
    {"$merge": {"into": temp_combined_collection}}
], allowDiskUse=True)

# Combined count of verified followers and followees
combined_verified_query = [
    {
        "$group": {
            "_id": "$id",
            "count": {"$sum": 1}
        }
    },
    {
        "$group": {
            "_id": None,
            "count": {"$sum": "$count"}
        }
    }
]

# Execute queries
verified_followers_count = database[followers_collection_name].aggregate(verified_followers_query).next()["count"]
verified_followees_count = database[followees_collection_name].aggregate(verified_followees_query).next()["count"]
result = database[temp_combined_collection].aggregate(combined_verified_query)

# Print the results
print("Number of Verified Followers:", verified_followers_count)
print("Number of Verified Followees:", verified_followees_count)
for doc in result:
    combined_verified_count = doc["count"]
    print("Number of Verified Followers and Followees Combined:", combined_verified_count)

# Drop the temporary collection
database[temp_combined_collection].drop()

# Close the MongoDB connection
client.close()

Number of Verified Followers: 1
Number of Verified Followees: 11
Number of Verified Followers and Followees Combined: 12


### VI

In [66]:
# Connect to MongoDB
client = MongoClient(mongo_uri)
database = client[database_name]

# Average favorites count query for followers
average_favorites_followers_query = [
    {
        "$group": {
            "_id": None,
            "average_favorites_count_followers": {"$avg": "$favourites_count"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_favorites_count_followers": {"$round": ["$average_favorites_count_followers", 2]}
        }
    }
]

# Average favorites count query for followees
average_favorites_followees_query = [
    {
        "$group": {
            "_id": None,
            "average_favorites_count_followees": {"$avg": "$favourites_count"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_favorites_count_followees": {"$round": ["$average_favorites_count_followees", 2]}
        }
    }
]

# Create a temporary collection by merging followers and followees
temp_combined_collection = "temp_combined_collection"
database[followers_collection_name].aggregate([
    {"$merge": {"into": temp_combined_collection}}
], allowDiskUse=True)

database[followees_collection_name].aggregate([
    {"$merge": {"into": temp_combined_collection}}
], allowDiskUse=True)

# Average favorites count query for combined followers and followees
average_favorites_combined_query = [
    {
        "$group": {
            "_id": None,
            "average_favorites_count_combined": {"$avg": "$favourites_count"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_favorites_count_combined": {"$round": ["$average_favorites_count_combined", 2]}
        }
    }
]

# Execute queries
result_followers = database[followers_collection_name].aggregate(average_favorites_followers_query)
result_followees = database[followees_collection_name].aggregate(average_favorites_followees_query)
result_combined = database[temp_combined_collection].aggregate(average_favorites_combined_query)

# Print the results
for doc in result_followers:
    print("Average Favorites Count of Followers:", doc["average_favorites_count_followers"])

for doc in result_followees:
    print("Average Favorites Count of Followees:", doc["average_favorites_count_followees"])

for doc in result_combined:
    print("Combined Average Favorites Count:", doc["average_favorites_count_combined"])

# Drop the temporary collection
database[temp_combined_collection].drop()

# Close the MongoDB connection
client.close()

Average Favorites Count of Followers: 1899.68
Average Favorites Count of Followees: 6092.41
Combined Average Favorites Count: 4317.81


### VII

In [67]:
# Connect to MongoDB
client = MongoClient(mongo_uri)
database = client[database_name]

# Average number of tweets query for followers
average_tweets_followers_query = [
    {
        "$group": {
            "_id": None,
            "average_tweets_count_followers": {"$avg": "$statuses_count"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_tweets_count_followers": {"$round": ["$average_tweets_count_followers", 2]}
        }
    }
]

# Average number of tweets query for followees
average_tweets_followees_query = [
    {
        "$group": {
            "_id": None,
            "average_tweets_count_followees": {"$avg": "$statuses_count"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_tweets_count_followees": {"$round": ["$average_tweets_count_followees", 2]}
        }
    }
]

# Create a temporary collection by merging followers and followees
temp_combined_collection = "temp_combined_collection"
database[followers_collection_name].aggregate([
    {"$merge": {"into": temp_combined_collection}}
], allowDiskUse=True)

database[followees_collection_name].aggregate([
    {"$merge": {"into": temp_combined_collection}}
], allowDiskUse=True)

# Average number of tweets query for combined followers and followees
average_tweets_combined_query = [
    {
        "$group": {
            "_id": None,
            "average_tweets_count_combined": {"$avg": "$statuses_count"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_tweets_count_combined": {"$round": ["$average_tweets_count_combined", 2]}
        }
    }
]

# Execute queries
result_followers = database[followers_collection_name].aggregate(average_tweets_followers_query)
result_followees = database[followees_collection_name].aggregate(average_tweets_followees_query)
result_combined = database[temp_combined_collection].aggregate(average_tweets_combined_query)

# Print the results
for doc in result_followers:
    print("Average Tweets Count of Followers:", doc["average_tweets_count_followers"])

for doc in result_followees:
    print("Average Tweets Count of Followees:", doc["average_tweets_count_followees"])

for doc in result_combined:
    print("Combined Average Tweets Count:", doc["average_tweets_count_combined"])

# Drop the temporary collection
database[temp_combined_collection].drop()

# Close the MongoDB connection
client.close()

Average Tweets Count of Followers: 654.21
Average Tweets Count of Followees: 4055.74
Combined Average Tweets Count: 2616.02


### VIII

In [98]:
# Connect to MongoDB
client = MongoClient(mongo_uri)
database = client[database_name]

# Extract the number of followers per year (handling different data types)
followers_per_year_query = [
    {
        "$match": {
            "creation_time": {"$exists": True}
        }
    },
    {
        "$group": {
            "_id": {"$year": {"$dateFromString": {"dateString": "$creation_time"}}},
            "count": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "year": "$_id",
            "Number of followers": "$count"
        }
    },
    {
        "$sort": {"year": 1}
    }
]

# Extract the number of followees per year (handling different data types)
followees_per_year_query = [
    {
        "$match": {
            "creation_time": {"$exists": True}
        }
    },
    {
        "$group": {
            "_id": {"$year": {"$dateFromString": {"dateString": "$creation_time"}}},
            "count": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "year": "$_id",
            "Number of followees": "$count"
        }
    },
    {
        "$sort": {"year": 1}
    }
]

# Execute queries
result_followers_per_year = database[followers_collection_name].aggregate(followers_per_year_query)
result_followees_per_year = database[followees_collection_name].aggregate(followees_per_year_query)

# Print the results for followers
print("\nyear | Number of followers")
for doc in result_followers_per_year:
    year = int(doc['year']) if doc['year'] is not None else 'None'
    print(f"{year} | {doc['Number of followers']}")

# Print the results for followees
print("\nyear | Number of followees")
for doc in result_followees_per_year:
    year = int(doc['year']) if doc['year'] is not None else 'None'
    print(f"{year} | {doc['Number of followees']}")

# Close the MongoDB connection
client.close()


year | Number of followers
None | 91

year | Number of followees
None | 124
